In [1]:
#This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

import xgboost as xgb
from sklearn.metrics import mean_squared_log_error as MSLE

# Any results you write to the current directory are saved as output.

# PPP, Salary and XGBoost
## 각 국의 Data scientists 수익을 분석해보는 커널
### PPP : 일물일가의 법칙을 전체적인 물가수준에 대해 확대 적용시킨 것
### 모든 물품에 대해서 비교할 순 없기 때문에, basket 이라는 개념을 이용해서 비교한다.

##### ex) 미국의 basket = 10USD, 인도의 basket = 200 INR
##### PPP(India) = 200/10 = 20
##### 위 예의 의미는 1 달러와 20 INR 이 같은 가치를 가진다는 것.

# Kagglers incomes adjusted according to PPPs
### 각 나라의 캐글러의 수입을 PPP rate 로 변환하고 살펴보자!
### 의미있는 결과를 보기 위해서 최소한 80명 이상이 응답한 데이터만 활용할 것이다.



In [2]:
data = pd.read_csv('../input/multipleChoiceResponses.csv', encoding="ISO-8859-1")

# data 에서 salaries 를 numerical value 로 바꾸고, 1,000 ~ 1,000,000 의 값들만 이용한다.
data['CompensationAmount'] = data['CompensationAmount'].fillna(0) # 결측값 채우기
data['CompensationAmount'] = data.CompensationAmount.apply(lambda x: 0 if (pd.isnull(x) or (x=='-') or (x==0))
                                                       else float(x.replace(',','')))
df = data[(data['CompensationAmount']>1000) & (data['CompensationAmount']<2000000)]


# 80명 이상의 응답자가 있는 국가만을 사용할것임.
s_temp = df['Country'].value_counts()
s_temp = s_temp[s_temp>80]
countries=list(s_temp.index)
countries.remove('Other')
df=df[df.Country.isin(countries)]

In [6]:
df.head()
df.count()

### Currency 열에 결측치가 있지만, 이를 삭제하지 않고 응답자가 살고있는 나라의 통화로 채울거다. 
###### 어떤 링크에 PPP rate 가 다 모여있었음.

In [3]:
df['CompensationCurrency'] =df.groupby('Country')['CompensationCurrency'].apply(lambda x: x.fillna(x.value_counts().idxmax()))

### 주어진 PPP rate 를 이용해서 Dataframe 을 만든다.

In [7]:
#The PPP rates
rates_ppp={'Countries':['United States','India','United Kingdom','Germany','France','Brazil','Canada','Spain','Australia','Russia','Italy',"People 's Republic of China",'Netherlands'],
           'Currency':['USD','INR','GBP','EUR','EUR','BRL','CAD','EUR','AUD','RUB','EUR','CNY','EUR'],
           'PPP':[1.00,17.7,0.7,0.78,0.81,2.05,1.21,0.66,1.46,25.13,0.74,3.51,0.8]}

rates_ppp = pd.DataFrame(data=rates_ppp)
rates_ppp

### 응답자들이 대부분 그 나라의 화폐를 사용하지만, 늘 그런것은 아니기 때문에  PPP rate 를 바로 이용할 수 없다.
### 그래서 다음의 과정을 거친다.
#### 1, kaggle 에서 주어진 시장환율을 이용해서 모든 소득을 USD 로 바꾼다.
#### 2, PPP / MER (Market Exchange Rate) 를 구한다.
#### 3, 비율 조정 계수 (PPP/MER) 을 이용해서 응답자들의 수입을 조정한다.

In [15]:
# 환율 데이터를 불러오기
rates_mer=pd.read_csv('../input/conversionRates.csv', encoding="ISO-8859-1")
rates_mer.drop('Unnamed: 0',inplace=True,axis=1)
# Unnamed: 0 이라는 열이 있음, axis =1 : 열 삭제

In [16]:
# 환율 데이터를 불러오기
rates_mer=pd.read_csv('../input/conversionRates.csv', encoding="ISO-8859-1")
rates_mer.drop('Unnamed: 0',inplace=True,axis=1)
# Unnamed: 0 이라는 열이 있음, axis =1 : 열 삭제

rates=rates_ppp.merge(rates_mer,left_on='Currency',right_on='originCountry',how='left')
# rates_ppp 와 rates_mer 을 합치는데, rates_ppp의 Currency 와 rates_mer 의 originCountry 를 가지고 left(Currency)로 합침
rates['PPP/MER']=rates['PPP']*rates['exchangeRate']

# 위 데이터들을 결합하기 
rates=rates[['Countries','PPP','PPP/MER']]
rates

In [17]:
df

In [18]:
rates_mer

In [19]:
rates

In [24]:
df.head()

In [30]:
# 데이터들을 합치는 과정
df=df.merge(rates_mer,left_on='CompensationCurrency',right_on='originCountry',how='left')
df=df.merge(rates,left_on='Country',right_on='Countries',how='left')
df['AdjustedSalary']=df['CompensationAmount']*df['exchangeRate']/df['PPP/MER']

# 중앙값 구하기
d_salary = {}
for country in df['Country'].value_counts().index :
    d_salary[country]=df[df['Country']==country]['AdjustedSalary'].median()
    
median_wages = pd.DataFrame.from_dict(data=d_salary, orient='index').round(2)
median_wages.sort_values(by=list(median_wages),axis=0, ascending=True, inplace=True)
ax = median_wages.plot(kind='barh',figsize=(15,8),width=0.7,align='center')
ax.legend_.remove()
ax.set_title("Adjusted incomes over the world",fontsize=16)
ax.set_xlabel("Amount", fontsize=14)
ax.set_ylabel("Country", fontsize=14)
for tick in ax.get_xticklabels():
    tick.set_rotation(0)
    tick.set_fontsize(10)
plt.tight_layout()

plt.show();

### 위 plot을 보면 인도는 33k USD, 중국은 50k USD  의 조정된 수익값을 가지는데, 이는 여전히 미국과 차이가 크다.
### 왜그럴까..
#### 이유1, 인구가 많은 나라 또는 개도국의 통화가치를 미국 달러보다 낮게 책정하는 경향이 있다.
#### 이유2, PPP rate 의 한계 때문이다. 
PPP rate 는 basket 을 이용한 값인데, 각 나라별로 완전히 같은 제품이 아닌 유사한 제품군을 사용한다.

# Which country values its Data Scientists the most ?

### 우리는 salary 의 중앙값을 알고 있다. 이 값과 각 나라의 다른 직군들의 수입 중앙값을 이용해서 얼마나 data scientists 가 많은 수익을 얻는지를 볼거다.

### 인터넷에 각 나라 사람들의 수입 평균이 있지만, 평균은 이상점에 로버스트하지 못하기 때문에 그 값을 사용할 순 없다.
### 그래서 갤럽에서 조사한 중앙값(2007~2012 동안의 값을 2013년에 발표)을 사용할 것이다.
### 화폐의 시간적 가치가 다르기 때문에 이를 조정해서 사용할 예정이다.
#### 1,  우선 2013년에 발표한 중앙값을 가져온다.
#### 2, 2013년에서 부터 지금까지의 인플레이션률을 본다.
#### 3, 인플레이션률을 적용한 조정된 중앙값을 구하고 이를 이용한다.


In [31]:
inflations={'Countries':['United States','India','United Kingdom','Germany','France','Brazil','Canada','Spain','Australia','Russia','Italy',"People 's Republic of China",'Netherlands'],
           'CPI_2013':[106.83,131.98,110.15,105.68,105.01,119.37,105.45,107.21,107.70,121.64,107.20,111.16,107.48],
           'CPI_2017':[113.10,162.01,116.51,109.6,107.1,156.73,112.39,109.13,113.48,168.50,108.61,119.75,111.55],
           'medians_2013':[15480,615,12399,14098,12445,2247,15181,7284,15026,4129,6874,1786,14450]}

rates_inflations = pd.DataFrame(inflations)
rates_inflations['adjusted_medians']=(rates_inflations['medians_2013']*rates_inflations['CPI_2017']/rates_inflations['CPI_2013']).round(2)
rates_inflations

### 이제 비교 할 준비가 되었으니 비교를 해봅시다.

In [32]:
tmp=median_wages.reset_index()
tmp = tmp.rename(columns={'index': 'Country', 0: 'median_income'})

rates_inflations=rates_inflations.merge(tmp,left_on='Countries',right_on='Country',how='left')
rates_inflations['ratio_incomes']=(rates_inflations['median_income']/rates_inflations['adjusted_medians']).round(2)

tmp2=rates_inflations[['Country','ratio_incomes']]
tmp2.sort_values(by='ratio_incomes',axis=0, ascending=True, inplace=True)

In [33]:
tmp2.plot.barh(x='Country',figsize=(12,8))
plt.show();

### 위 plot 을 보면 인도에서는 Data scientists 가 44배 많은 수입을 받는다는 것을 알 수 있다. (중국은 25배, 브라질은 15배)
### 하지만 위와 같은 BRICS 국가들은 빠르게 성장 중이지만, 높은 임금격차와 빈곤률을 보인다. 
### 이러한 사회적, 경제적 불균형 때문에 위와 같은 결과가 나타난 것이다.
### 다른 선진국들을 보면 평균적인 비율을 보이고 있다.


### 다음으로는 우리는 각 국의 Data scientists 의 수입에 가장 큰 영향을 미치는 요소를 보기 위해 좀 더 일반적인 것들을 살펴볼 예정이다.

# Who are the most valued Data Scientists ?

### 여기서는Data scientist 가 어떤 프로그램을 쓰는지, 어떤 역할을 맡고 있는지, 그들이 가장 자주 쓰는 ML/DS 가 무엇인지에 따라 얼마나 수익을 거두는지에 대해 볼 것이다.

### 유의미한 결과를 얻기 위해서, 급여가 비슷한 국가들 끼리 basket 을 구성한다.

#### 1, 첫번째 그룹은 US 만
#### 2, 두번째 그룹은 Australia, Netherlands, Germany, United Kingdom and Canada
#### 3, 세번째 그룹은 Spain, France, Italy, China, Brazil
#### 4, 네번째 그룹은  India and Russia






In [34]:
datasets = {'USA' : df[df['Country']=='United States'] , 
            'Eur+Ca' :df[df.Country.isin(['Australia','Germany','Canada','United Kingdom','Netherlands'])],
            'Eur2+Bra+Chi' : df[df.Country.isin(['Spain','France','Brazil',"People 's Republic of China",'Italy'])],
            'India/Russia' : df[df.Country.isin(['India','Russia'])]}

In [35]:
methods=['WorkMethodsFrequencyBayesian','WorkMethodsFrequencyNaiveBayes','WorkMethodsFrequencyLogisticRegression',
       'WorkMethodsFrequencyDecisionTrees','WorkMethodsFrequencyRandomForests',
       'WorkMethodsFrequencyEnsembleMethods','WorkMethodsFrequencyDataVisualization','WorkMethodsFrequencyPCA',
       'WorkMethodsFrequencyNLP','WorkMethodsFrequencyNeuralNetworks',
       'WorkMethodsFrequencyTextAnalysis',
       'WorkMethodsFrequencyRecommenderSystems','WorkMethodsFrequencyKNN','WorkMethodsFrequencySVMs',
       'WorkMethodsFrequencyTimeSeriesAnalysis']

d_method_countries={} 
for key, value in datasets.items():
    d_method_countries[key]={}
    for col in methods : 
        method = col.split('WorkMethodsFrequency')[1]
        d_method_countries[key][method]=value[value[col].isin(['Most of the time','Often'])]['AdjustedSalary'].median()
        
positions=[(0,0),(1,0),(0,1),(1,1)]
f,ax=plt.subplots(nrows=2, ncols=2,figsize=(15,8))
for ((key, value), pos) in zip(d_method_countries.items() , positions):
    methods = pd.DataFrame.from_dict(data=value, orient='index').round(2)
    methods.sort_values(by=list(methods),axis=0, ascending=True, inplace=True)
    methods.plot(kind='barh',figsize=(12,8),width=0.7,align='center',ax=ax[pos[0],pos[1]])
    ax[pos[0],pos[1]].set_title(key,fontsize=14)
    ax[pos[0],pos[1]].legend_.remove()
    

plt.tight_layout()
plt.show();



### 위의 plot 을 해석해 보면, 국가에 관계없이 Recommender System 을 사용하는 사람들은 높은 봉급을 받았다.
### 그리고 US, Canada and the first group of european countries 에서는 Naive Bayes 가 가장 높은 순위를 차지했지만,
### BRICS countries (see above) + other european countries 에서는 가장 낮은 순위를 차지했다.
### 다른 항목들도 보면 조금의 차이를 보인다. ex) Data Visualization

# Tools and programming languages used at work

In [36]:
tools=['WorkToolsFrequencyC','WorkToolsFrequencyJava','WorkToolsFrequencyMATLAB',
       'WorkToolsFrequencyPython','WorkToolsFrequencyR','WorkToolsFrequencyTensorFlow',
       'WorkToolsFrequencyHadoop','WorkToolsFrequencySpark','WorkToolsFrequencySQL',
       'WorkToolsFrequencyNoSQL','WorkToolsFrequencyExcel','WorkToolsFrequencyTableau',
       'WorkToolsFrequencyJupyter','WorkToolsFrequencyAWS',
       'WorkToolsFrequencySASBase','WorkToolsFrequencyUnix']

d_tools_countries={} 
for key, value in datasets.items():
    d_tools_countries[key]={}
    for col in tools : 
        tool = col.split('WorkToolsFrequency')[1]
        d_tools_countries[key][tool]=value[value[col].isin(['Most of the time','Often'])]['AdjustedSalary'].median()
        
positions=[(0,0),(1,0),(0,1),(1,1)]
f,ax=plt.subplots(nrows=2, ncols=2,figsize=(15,8))
for ((key, value), pos) in zip(d_tools_countries.items() , positions):
    tools = pd.DataFrame.from_dict(data=value, orient='index').round(2)
    tools.sort_values(by=list(methods),axis=0, ascending=True, inplace=True)
    tools.plot(kind='barh',figsize=(12,8),width=0.7,align='center',ax=ax[pos[0],pos[1]])
    ax[pos[0],pos[1]].set_title(key,fontsize=14)
    ax[pos[0],pos[1]].legend_.remove()
    

plt.tight_layout()
plt.show();

### 이 plot 을 보면 Hadoop, Spark, AWS 등 대용량 데이터 아키텍쳐와 함께 사용되는 기술은 어디에서나 높은 봉급을 받습니다.

# Job titles

In [37]:
titles=list(df['CurrentJobTitleSelect'].value_counts().index)
d_titles_countries={} 
for key, value in datasets.items():
    d_titles_countries[key]={}
    for title in titles : 
        d_titles_countries[key][title]=value[value['CurrentJobTitleSelect']==title]['AdjustedSalary'].median()
        
positions=[(0,0),(1,0),(0,1),(1,1)]
f,ax=plt.subplots(nrows=2, ncols=2,figsize=(15,8))
for ((key, value), pos) in zip(d_titles_countries.items() , positions):
    tools = pd.DataFrame.from_dict(data=value, orient='index').round(2)
    tools.sort_values(by=list(methods),axis=0, ascending=True, inplace=True)
    tools.plot(kind='barh',figsize=(12,8),width=0.7,align='center',ax=ax[pos[0],pos[1]])
    ax[pos[0],pos[1]].set_title(key,fontsize=14)
    ax[pos[0],pos[1]].legend_.remove()
    

plt.tight_layout()
plt.show();

### 위 plot을 보면.... business analysts and data analysts, data scientists or ML engineers
### 국가별로 다른 needs 를 가지고 있다는 것을 알 수 있음.


# Job Function

In [38]:
func = list(df['JobFunctionSelect'].value_counts().index)
tmp = df
tmp=tmp.replace(to_replace=func, value=['Analyze data','Build a ML service','Build prototypes',
                                        'Build the Data Infrastructure','Other','Research'])

datasets_tmp = {'USA' : tmp[tmp['Country']=='United States'] , 
            'Eur+Ca' :tmp[tmp.Country.isin(['Australia','Germany','Canada','United Kingdom','Netherlands'])],
            'Eur2+Bra+Chi' : tmp[tmp.Country.isin(['Spain','France','Brazil',"People 's Republic of China",'Italy'])],
            'India/Russia' : tmp[tmp.Country.isin(['India','Russia'])]}

functions=list(tmp['JobFunctionSelect'].value_counts().index)
d_functions_countries={} 
for key, value in datasets_tmp.items():
    d_functions_countries[key]={}
    for function in functions : 
        d_functions_countries[key][function]=value[value['JobFunctionSelect']==function]['AdjustedSalary'].median()
        
positions=[(0,0),(1,0),(0,1),(1,1)]
f,ax=plt.subplots(nrows=2, ncols=2,figsize=(15,8))
for ((key, value), pos) in zip(d_functions_countries.items() , positions):
    tools = pd.DataFrame.from_dict(data=value, orient='index').round(2)
    tools.sort_values(by=list(methods),axis=0, ascending=True, inplace=True)
    tools.plot(kind='barh',figsize=(15,8),width=0.7,align='center',ax=ax[pos[0],pos[1]])
    ax[pos[0],pos[1]].set_title(key,fontsize=14)
    ax[pos[0],pos[1]].legend_.remove()
    
plt.tight_layout()
plt.show();

### 한번 볼것들. Building prototypes, researcher 

# How hard is it to predict the incomes for different respondents ?

### 이번에는 kaggler 들이 어느정도의 연봉을 받을지에 대해 예측을 해 볼 것이다.

In [40]:
df['MATLABUsers']=[1 if df['WorkToolsFrequencyMATLAB'].iloc[i] in ['Most of the time','Often'] else 0 for i in range(df.shape[0])]
df['AWSUsers']=[1 if df['WorkToolsFrequencyAWS'].iloc[i] in ['Most of the time','Often'] else 0 for i in range(df.shape[0])]
df['HadoopUsers']=[1 if df['WorkToolsFrequencyHadoop'].iloc[i] in ['Most of the time','Often'] else 0 for i in range(df.shape[0])]
df['SparkUsers']=[1 if df['WorkToolsFrequencySpark'].iloc[i] in ['Most of the time','Often'] else 0 for i in range(df.shape[0])]

df['NaiveBayesUsers']=[1 if df['WorkMethodsFrequencyNaiveBayes'].iloc[i] in ['Most of the time','Often'] else 0 for i in range(df.shape[0])]
df['RecommenderSystemsUsers']=[1 if df['WorkMethodsFrequencyRecommenderSystems'].iloc[i] in ['Most of the time','Often'] else 0 for i in range(df.shape[0])]
df['DataVisualizationUsers']=[1 if df['WorkMethodsFrequencyDataVisualization'].iloc[i] in ['Most of the time','Often'] else 0 for i in range(df.shape[0])]

features= ['GenderSelect','Country','Age','FormalEducation','MajorSelect','ParentsEducation',
           'EmploymentStatus','StudentStatus','DataScienceIdentitySelect','CodeWriter',
           'CurrentEmployerType','SalaryChange','RemoteWork','WorkMLTeamSeatSelect',
           'Tenure','EmployerIndustry','EmployerSize','CurrentJobTitleSelect','JobFunctionSelect',
           'MATLABUsers','AWSUsers','HadoopUsers','SparkUsers',
           'NaiveBayesUsers','RecommenderSystemsUsers','DataVisualizationUsers',
           'TimeGatheringData','TimeModelBuilding','TimeProduction','TimeVisualizing','TimeFindingInsights',
           'AdjustedSalary']

### 같은 결과가 나오지 않게 하기 위해서 그룹화를 다시 한다.

#### 1, US
#### 2, BRICS 국가
#### 3, 모든 유럽국가 + 캐나다, 호주

In [41]:
df_us = df[df['Country']=='United States'][features]
df_eur = df[df.Country.isin(['Spain','France','Germany','Canada','United Kingdom','Netherlands','Italy','Australia','Canada'])][features]
df_bric = df[df.Country.isin(['India','Russia','Brazil',"People 's Republic of China"])][features]

 ### 예측은 Mean Squared Logarithmic Error 을 이용해서 할 것이다.

In [42]:
for (dataset,zone) in zip([df_us,df_bric,df_eur],['USA','BRIC','Europe + Canada and Australia']) : 
    
    dataset=pd.get_dummies(dataset,columns=['GenderSelect','Country','FormalEducation','MajorSelect','ParentsEducation',
           'EmploymentStatus','StudentStatus','DataScienceIdentitySelect','CodeWriter',
           'CurrentEmployerType','SalaryChange','RemoteWork','WorkMLTeamSeatSelect',
           'Tenure','EmployerIndustry','EmployerSize','CurrentJobTitleSelect','JobFunctionSelect'])
    for col in ['Age','TimeGatheringData','TimeModelBuilding','TimeProduction','TimeVisualizing','TimeFindingInsights']:
        dataset[col] = dataset[col].fillna(value=dataset[col].median())
    dataset.dropna(axis=0,inplace=True)

    np.random.seed(42)
    perm = np.random.permutation(dataset.shape[0])
    train = dataset.iloc[perm[0:round(0.85*len(perm))]]
    test = dataset.iloc[perm[round(0.85*len(perm))::]]
    y_train , y_test = train['AdjustedSalary'] , test['AdjustedSalary']
    X_train , X_test = train.drop('AdjustedSalary',axis=1) , test.drop('AdjustedSalary',axis=1)

    clf=xgb.XGBRegressor(learning_rate=0.05, n_estimators=500, objective='reg:linear',reg_lambda=0.5, 
                         random_state=42)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    
    print('Prediction for : %s'%zone)
    print('The RMSLE score is : {:0.4f}'.format(np.sqrt(MSLE(y_test,y_pred)) /np.sqrt(len(y_pred)) ))
    print('-------------------------------------------------')

### 나쁘지 않을 결과를 얻었다고 한다...

### 그렇다면 분포도를 한번 보자!

In [43]:
f,ax=plt.subplots(nrows=1, ncols=3,figsize=(15,8))
df_bric['AdjustedSalary'].plot.hist(bins=50,ax=ax[0],figsize=(15,8),title='Salary distribution in BRIC countries')
df_eur['AdjustedSalary'].plot.hist(bins=50,ax=ax[1],figsize=(15,8),title='Salary distribution in Europe + Ca+ Aus')
df_us['AdjustedSalary'].plot.hist(bins=50,ax=ax[2],figsize=(15,8),title='Salary distribution in the the US')
plt.show();

### 미국과 유럽국가들의 분포는 거의 비슷했다. 아주 높은 value 때문에 약간 오른쪽으로 치우친 모습을 보인다.
### BRICS 국가들은 오른쪽으로 크게 치우쳐있으며 수입의 대부분의 value 가 0 이다.
### BRICS 국가들은 다른 국가들에 비해 이상점의 bin 이 빨리 나올 것이다. Boxplot 을 한번 보자!

In [44]:
f,ax=plt.subplots(nrows=1, ncols=3,figsize=(15,8))
sns.boxplot(y=df_bric['AdjustedSalary'],data=df_bric,ax=ax[0]).set_title('Quartiles and outliers in BRIC')
sns.boxplot(y=df_eur['AdjustedSalary'],data=df_eur,ax=ax[1]).set_title('Quartiles and outliers in EUR')
sns.boxplot(y=df_us['AdjustedSalary'],data=df_us,ax=ax[2]).set_title('Quartiles and outliers in USA')
plt.show();

### BRICS 국가의 boxplot 을 보면 0 가까이에 많은 값들이 있고 이상점들도 빨리 나타난 것을 볼 수 있다. 
### 이러한 점들은 BRICS 의 데이터가 충분하지 않았기 때문이라고 생각한다.

# Conclusion and perspectives 

### 데이터와 도메인지식을 활용해서 분석을 잘 해보도록 하자